In [10]:
# 假设我们想对函数 y=2x^Tx 关于列向量x求导
import torch

x = torch.arange(4.0)
x

tensor([0., 1., 2., 3.])

In [11]:
# 在我们计算 y 关于 x 的梯度之前, 我们需要一个地方来存储梯度
x.requires_grad_(True)  # 等价于 `x = torch.arange(4.0, requires_grad=True)` 
x.grad  # 默认值是 None

In [12]:
# 现在让我们计算 y 
y = 2 * torch.dot(x, x)
y

tensor(28., grad_fn=<MulBackward0>)

In [13]:
# 通过调用反向传播函数来自动计算 y 关于 x 每个分量的梯度
y.backward()
x.grad

tensor([ 0.,  4.,  8., 12.])

In [14]:
x.grad == 4 * x

tensor([True, True, True, True])

In [16]:
# 现在让我们计算 x 的另一个函数
# 在默认情况下, PyTorch会累积梯度, 我们需要清除之前的值
x.grad.zero_()
y = x.sum()
y.backward()
x.grad

tensor([1., 1., 1., 1.])

In [17]:
# 深度学习中, 我们的目的不是计算微分矩阵, 而是批量中每个样本单独计算的偏导数之和
x.grad.zero_()
y = x * x
y.sum().backward()
x.grad

tensor([0., 2., 4., 6.])

In [21]:
# 将某些计算移动到记录的计算图之外
x.grad.zero_()
y = x * x
u = y.detach()
z = u * x

z.sum().backward()
x.grad == u

tensor([True, True, True, True])

In [22]:
x.grad.zero_()
y.sum().backward()
x.grad == 2 * x

tensor([True, True, True, True])

In [23]:
# 即使构建函数的计算图需要通过python控制流(例如, 条件、循环或任意函数调用), 我们仍然可以计算得到变量的梯度
def f(a):
    b = a * 2
    while b.norm() < 1000:
        b = b * 2
    if b.sum() > 0:
        c = b
    else:
        c = 100 * b
    return c

a = torch.randn(size=(), requires_grad=True)
d = f(a)
d.backward()

a.grad == d / a

tensor(True)